In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

import torchvision
import torchvision.transforms as transforms

In [4]:
torch.set_printoptions(linewidth=120)

In [5]:
class MyNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):

        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.fc1(t.reshape(-1, 12*4*4)))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        
        return t

In [6]:
train_set = torchvision.datasets.FashionMNIST(
root='./data/FashionMNIST', train=True, download=True,transform=transforms.Compose([
    transforms.ToTensor()
])) 


def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [13]:
network = MyNetwork()
optimizer = optim.Adam(network.parameters(),lr=0.01)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
batch = next(iter(train_loader))

In [ ]:
epoch_max = 5
epoch = 0
for epoch in range(5):
    total_loss = 0
    total_correct = 0

    for batch in train_loader:
        images, labels = batch

        preds = network(images)
        loss = F.cross_entropy(preds, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss +=loss.item()
        total_correct += get_num_correct(preds, labels)
    print('epoch:', epoch,'total_correct:',total_correct, 'total_loss:',total_loss)

epoch: 0 total_correct: 47967 total_loss: 318.94849337637424
epoch: 1 total_correct: 51720 total_loss: 223.413066893816
epoch: 2 total_correct: 52429 total_loss: 204.05664820969105
epoch: 3 total_correct: 52657 total_loss: 195.52699391543865


In [ ]:
total_correct/len(train_set)